## Start by downloading all the hydrosheds flow direction tiles and merging - I found it easiest to merge the tiles in qgis (merge function), or arcgis.  I did this for australia and now have the combined file aus_flowdir.tif which is used below

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
from numba import jit
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Open file using rasterio

In [18]:
f = rio.open('aus_flowdir.tif')
#a = f.read()
#a = a.squeeze() # Need to squeeze to remove extra dimension

## The area we are interested in, is actually a bit smaller than the whole area of the geotiff, which includes Australia

In [56]:
a_sub = a[0:35000,0:51000]  

## Need to transform the direction format so that each cell gives the linear index of its downstream neighbor.  The current format is set up so that it only gives the cardinal directions indicated by 0, 255 = outlets (0 ocean, 255 internal), 1 = east draining, then rotate clockwise in the d8 format 2 SE,4 S,8 SW,16 W ,32 NW,64 N, 128 NE.

In [60]:
from numba import jit
@jit(nopython=True)

def h_flowdir(d):
    #Translate flow dir from hydrosheds into flow dir from simplem format
    ny, nx = np.shape(d)
    s = np.zeros(np.shape(d)) #slopes formatted correctly for simplem
    for i in range(ny):
        for j in range(nx):
            d1 = d[i, j]
            if (d1 == 0) or (d1 == 255): #0 is outlet to ocean, 255 is internally drained pour point
                s[i, j] = j * ny + i
            elif d1 == 1:
                s[i, j] = j * ny + i + ny
            elif d1 == 2:
                s[i, j] = j * ny + i + ny + 1
            elif d1 == 4:
                s[i, j] = j * ny + i + 1
            elif d1 == 8:
                s[i, j] = j * ny +i - ny + 1
            elif d1 == 16:
                s[i, j] = j * ny  +i- ny
            elif d1 == 32:
                s[i, j] = j * ny +i - ny - 1
            elif d1 == 64:
                s[i, j] = j * ny + i - 1
            elif d1 == 128:
                s[i, j] = j * ny + i - 1 + ny
            
    return s
                               


In [61]:
r = h_flowdir(a_sub)

## From the receiver matrix r, we can build the topologic "stack" .  See the guide in github.com/ruetg/Simplem to see how this is done.  I've copied the simplem stack function here.

In [6]:
import math
@jit(nopython=True)
def lind(xy,n):
    #Compute linear index from 2 points
    x=math.floor(int(xy)/n)
    y=xy%n
    return int(y),int(x)

@jit(nopython=True)
def stack(s):
    #This takes the input s and makes the topologic stack of the stream network in O(n) time
    c=0
    k=0
    ny,nx = np.shape(s)
    I = np.zeros(ny*nx)
    for i in range(ny):
        for j in range(nx):

            ij = j * ny + i
            i2 = i
            j2 = j
            if s[i, j] == ij:

                I[c]=ij
                c+=1

                while c>k and c < ny * nx - 1 :
                    for i1 in range(-1, 2):
                        for j1 in range(-1, 2):
                                if j2 + j1 > 0 and i2 + i1 > 0 and j2 + j1 < nx - 1 and i2 + i1 < ny - 1:
                                    ij2 = (j2 + j1) * ny + i2 + i1
                                    #print(s[i2 + i1, j2 + j1])

                                    if ij != ij2 and s[int(i2 + i1), int(j2 + j1)] == ij:
                                        I[c] = ij2
                                        c+=1
   
                                        #print(i1)
                                        #print(j1)


                    k=k+1
                    ij=I[k]
                    i2,j2=lind(ij,ny)
        if np.mod(i,1000) == 0:
            print(i/ny)
    return I


In [ ]:
I = stack(r)

## At this point it may have been enough computation time to justify saving the stack and receiver vectors in case of a crash ... 

In [64]:
I = np.int32(I)
r = np.int32(r)

np.save('stack_r',r)
np.save('stack_I',I)

In [3]:
#r = np.load('stack_r.npy')
#I = np.load('stack_I.npy')

## Compute the drainage area in units of cells summing downstream along the stack

In [8]:
@jit(nopython=True)
def acc(s,I):#Calculate drainage area
    A = np.ones(np.shape(s))
    ny,nx = np.shape(s)
    for ij in range(len(I)-1,0,-1):
        i,j=lind(I[ij],ny)
        i2,j2=lind(int(s[i,j]),ny)
        if I[ij] != s[i,j]:
            A[i2,j2] += A[i,j]
    return A

In [9]:
A = acc(r,I)

## Now can save the drainage area as geotiff

In [20]:
w = f.profile

In [21]:
ny,nx = np.shape(A)

In [22]:
w['width'] = nx
w['height'] = ny


In [73]:
with rio.open('acc.tif', 'w', **w) as dst:
    dst.write(A.astype(rio.int32), 1)

In [23]:
f.close()


## Chi is calculated based exclusively on drainage area - see Willett et al., 2014 for eqns.

In [10]:
@jit(nopython=True)
def chicalc(s,I,A):
    m = .5 # concavity for SP equation, may need to be modified
    dx = dy = 90 # Meters, approximate cell size
    ny, nx = np.shape(r)
    chi = np.zeros((ny, nx))

    dA= (dx * dy) ** m

    for ij in range(len(I)):
        i, j = lind( I[ij], ny)
        i2, j2= lind( s[i,j], ny)
        chi[i, j] = 1 / (A[i, j] ** m * dA)
        chi[i, j] += chi[i2, j2]
    return chi
        

In [15]:
chi = chicalc(r,I,A)

In [32]:
chi*=10000

In [33]:
w['dtype'] = 'int32'
with rio.open('chi.tif', 'w', **w) as dst:
    dst.write(chi.astype(rio.int32), 1)

In [29]:
np.min(chi)

0.0